# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
pip install pyparsing


In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,290.07,93,74,1.40,IN,1700263068
1,1,grytviken,-54.2811,-36.5092,272.45,74,18,0.72,GS,1700263068
2,2,ta`u,-14.2336,-169.5144,301.11,83,100,7.98,AS,1700263069
3,3,hawaiian paradise park,19.5933,-154.9731,299.88,71,20,4.63,US,1700263069
4,4,port-aux-francais,-49.3500,70.2167,278.74,97,100,13.06,TF,1700263069


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
# Dependencies
import folium
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Normalize humidity values
scaler = MinMaxScaler(feature_range=(1, 10))  # Adjust the feature range for desired circle sizes
city_data_df['Normalized_Humidity'] = scaler.fit_transform(city_data_df[['Humidity']])

# Generate a categorical color palette
num_colors = len(city_data_df['City'].unique())
color_palette = sns.color_palette("husl", n_colors=num_colors)

# Create a dictionary to map each city to a unique color
city_color_mapping = dict(zip(city_data_df['City'].unique(), color_palette))

# Configure the map plot
m = folium.Map(location=[0, 0], zoom_start=2)  # Set the initial location and zoom level

# Add a marker for each city with a unique color based on city name
for index, row in city_data_df.iterrows():
    color = city_color_mapping.get(row['City'], 'blue')  # Default to blue if city not in mapping
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Normalized_Humidity'],
        popup=f"{row['City']}, {row['Country']}\nHumidity: {row['Humidity']}%",
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)

# Display the map
m




### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Define your ideal weather conditions
ideal_conditions = (city_data_df['Max Temp'] < 290) & (city_data_df['Max Temp'] > 150) 
                   #(city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 30)

# Apply the conditions to filter the DataFrame
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Normalized_Humidity
1,1,grytviken,-54.2811,-36.5092,272.45,74,18,0.72,GS,1700263068,7.483871
4,4,port-aux-francais,-49.3500,70.2167,278.74,97,100,13.06,TF,1700263069,9.709677
6,6,iqaluit,63.7506,-68.5145,273.00,69,100,8.23,CA,1700263070,7.000000
9,9,aykhal,66.0000,111.5000,251.48,100,99,1.33,RU,1700263070,10.000000
10,10,bethel,41.3712,-73.4140,288.44,80,100,0.89,US,1700262997,8.064516
...,...,...,...,...,...,...,...,...,...,...,...
533,533,sodra sandby,56.5667,16.6167,276.98,74,100,3.73,SE,1700263234,7.483871
534,534,ennis,52.8436,-8.9864,285.11,94,75,3.60,IE,1700263234,9.419355
538,538,al qaryatayn,34.2296,37.2407,287.27,76,0,1.04,SY,1700263238,7.677419
539,539,hemsby,52.6971,1.6918,281.52,98,100,4.38,GB,1700263239,9.806452


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
1,grytviken,GS,-54.2811,-36.5092,74,
4,port-aux-francais,TF,-49.3500,70.2167,97,
6,iqaluit,CA,63.7506,-68.5145,69,
9,aykhal,RU,66.0000,111.5000,100,
10,bethel,US,41.3712,-73.4140,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000  # Adjust the radius as needed
params = {
    "type": "hotel",
    "limit": 1,  # Limit the results to 1 hotel
    "apikey": geoapify_key  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{latitude},{longitude}"
    params["bias"] = f"circle:{radius}@{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
grytviken - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
glendive - nearest hotel: No hotel found
jakar - nearest hotel: No hotel found
waurn ponds - nearest hotel: No hotel found
nadym - nearest hotel: No hotel found
vyazemskiy - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
kuanda - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
kirensk - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
weinan - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
kez - nearest hotel: No hotel found
ioannina - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
blackmans bay - n

,City,Country,Lat,Lng,Humidity,Hotel Name
1,grytviken,GS,-54.2811,-36.5092,74,No hotel found
4,port-aux-francais,TF,-49.3500,70.2167,97,No hotel found
6,iqaluit,CA,63.7506,-68.5145,69,No hotel found
9,aykhal,RU,66.0000,111.5000,100,No hotel found
10,bethel,US,41.3712,-73.4140,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
# Configure the map plot
hotel_map = folium.Map(location=[0, 0], zoom_start=2)  # Set the initial location and zoom level

# Add a marker for each hotel with the corresponding color from the original city map
for index, row in hotel_df.iterrows():
    color = city_color_mapping.get(row['City'], 'blue')  # Default to blue if city not in mapping
    
    folium.Marker(
        location=[row['Lat'], row['Lng']],
        popup=f"{row['City']}, {row['Country']}\nHotel: {row['Hotel Name']}",
        icon=folium.Icon(color=color)
    ).add_to(hotel_map)

# Display the map
hotel_map

C:\Users\mitch\AppData\Local\Temp\ipykernel_2836\2923038163.py:11: UserWarning: color argument of Icon should be one of: {'beige', 'pink', 'cadetblue', 'darkgreen', 'darkred', 'lightred', 'blue', 'red', 'gray', 'black', 'darkpurple', 'lightgreen', 'white', 'lightblue', 'lightgray', 'orange', 'purple', 'green', 'darkblue'}.
  icon=folium.Icon(color=color)
